In [9]:
import numpy as np
import pickle
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import functools
from sklearn.model_selection import train_test_split
import csv
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import tensorflow as tf

print ("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.0.0-alpha0
Eager execution: True


In [2]:
#value that the model is intended to predict.

LABEL_COLUMN = 'Smoke_Detector'
LABELS = [0, 1]

train_file_path='data1-ts.csv'
test_file_path='data-old.csv'

#Function to read the CSV data from the file and create a test dataset
def get_dataset_test(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=66,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset
#Function to read the CSV data from the file and create a train dataset
def get_dataset_train(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=198,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=100,
      ignore_errors=True,
      **kwargs)
  return dataset

#read the CSV data from the file and create a train dataset
raw_train_data = get_dataset_train(train_file_path)
raw_test_data = get_dataset_test(test_file_path)

#Column names in dataset
SELECT_COLUMNS = ['Fire_Sprinkler', 'Building_Occupancy', 'Room_Name', 'Room_Area', 'Smoke_Detector']

#read the CSV data from the file and create a train dataset
train_dataset = get_dataset_train(train_file_path, select_columns=SELECT_COLUMNS)
test_dataset = get_dataset_test(test_file_path, select_columns=SELECT_COLUMNS)


In [3]:
#Continuous data to be normalized.
train_batch, labels_batch = next(iter(train_dataset)) 
test_batch, labels_batch = next(iter(test_dataset)) 

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['Room_Area']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

train_batch, labels_batch = next(iter(packed_train_data))
test_batch, labels_batch = next(iter(packed_test_data))
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()


MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

Instructions for updating:
Use `tf.cast` instead.


array([[-0.81132984],
       [-0.81132984],
       [-0.81132984],
       [ 0.35949966],
       [-0.2259151 ],
       [-0.81132984],
       [-0.81132984],
       [-0.81132984],
       [-0.81132984],
       [-0.2259151 ],
       [ 0.35949966],
       [ 0.35949966],
       [ 0.35949966],
       [-0.2259151 ],
       [-0.81132984],
       [-0.2259151 ],
       [-0.81132984],
       [-0.81132984],
       [-0.81132984],
       [-0.2259151 ],
       [-0.2259151 ],
       [-0.2259151 ],
       [ 0.35949966],
       [-0.81132984],
       [-0.2259151 ],
       [-0.81132984],
       [ 0.35949966],
       [-0.81132984],
       [ 0.35949966],
       [-0.2259151 ],
       [ 0.35949966],
       [-0.81132984],
       [-0.2259151 ],
       [ 0.35949966],
       [ 0.35949966],
       [-0.81132984],
       [-0.81132984],
       [ 0.35949966],
       [-0.81132984],
       [ 2.1157439 ],
       [ 3.2865736 ],
       [-0.2259151 ],
       [-0.81132984],
       [-0.2259151 ],
       [-0.2259151 ],
       [-0

In [4]:
#content in categories should be one of a limited set of options
CATEGORIES={}
CATEGORIES = {
     'Room_Name':[b'  Bedroom'  ,
 b'  Hall'  ,
 b'  Living Room'  ,
 b'  Closet'  ,
 b'  Kitchen'  ,
 b'  Master Bedroom'  ,
 b'  Guest Bedroom'  ,
 b'  Foyer'  ,
 b'  Breakfast'  ,
 b'  Den'  ,
 b'  Office'  ,
 b'  Playroom'  ,
 b'  Corridor'  ,
 b'  Elevator Lobby'  ,
 b'  Lobby'  ,
 b'  Electrical Room'  ,
 b'  Mechanical Room'  ,
 b'  Storage'  ,
 b'  Garage'  ,
 b'  Fire Pump Room'  ,
 b'  Generator Room'  ,
 b'  Gym'  ,
 b'  Procedure Room'  ,
 b'  Nurse Station'  ,
 b'  Reception'  ,
 b'  Exam Room'  ,
 b'  Equipment Room'  ,
 b'  MRI Room'  ,
 b'  Control Room'  ,
 b'  Operating Room'  ,
 b'  Conference Room'  ,
 b'  Break Room'  ,
 b'  Bathroom'  ,
 b'  Locker Room'  ,
 b'  Nuc Med Room'  ,
 b'  Spect CT Room'  ,
 b'  Pre/Post Op'  ,
 b'  Patient Bedroom'  ,
 b'  Conference'  ,
 b'  Breakroom'  ],
    'Building_Occupancy' : [b'  Apartment Building',b'  Medical Building',b'  Office Building'],
    'Fire_Sprinkler' : [b'Yes',b'No']
}

#add any missing room names in database not specified in categories
df = pd.read_csv('data1-ts.csv')
for i in df['Room_Name']:
    if i not in CATEGORIES['Room_Name']:
        CATEGORIES['Room_Name'].append(i)
        
#create a collection
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [5]:
#Room Area was determined to not affect the prediction accuracy
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
#preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)


In [6]:
#Build the model
model = tf.keras.Sequential([
  categorical_layer,
  #preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),  
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [7]:
#model can be instantiated and trained
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data.shuffle(500)
model.fit(train_data, epochs=3)

#check its accuracy on the test_data set
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Epoch 1/3
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
151/151 [==============================] - 7s 44ms/step - loss: 0.1814 - accuracy: 0.8429
Epoch 2/3
151/151 [==============================] - 4s 25ms/step - loss: 0.0379 - accuracy: 0.9822
Epoch 3/3
151/151 [==============================] - 4s 24ms/step - loss: 0.0370 - accuracy: 0.9823
      1/Unknown - 0s 18ms/step - loss: 0.1007 - accuracy: 0.9697

Test Loss 0.100699990988, Test Accuracy 0.969696998596


In [8]:
#infer labels on a batch or a dataset of batches
predictions = model.predict(test_data)

for prediction, smoke_detector in zip(predictions[:66], list(test_data)[0][1][:66]):
    print("Smoke Detector: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Smoke Detector" if bool(smoke_detector) else "No Smoke Detector"))
  


Smoke Detector: 99.97%  | Actual outcome:  Smoke Detector
Smoke Detector: 0.04%  | Actual outcome:  No Smoke Detector
Smoke Detector: 100.00%  | Actual outcome:  Smoke Detector
Smoke Detector: 0.03%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.61%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.98%  | Actual outcome:  Smoke Detector
Smoke Detector: 0.06%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.05%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.97%  | Actual outcome:  Smoke Detector
Smoke Detector: 0.03%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.95%  | Actual outcome:  Smoke Detector
Smoke Detector: 0.17%  | Actual outcome:  No Smoke Detector
Smoke Detector: 100.00%  | Actual outcome:  Smoke Detector
Smoke Detector: 99.95%  | Actual outcome:  Smoke Detector
Smoke Detector: 99.95%  | Actual outcome:  Smoke Detector
Smoke Detector: 0.00%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.14%  | Actual outcome:  No Smoke Det

In [31]:
#save predictions to list
li=(predictions)[:66].tolist()

[[0.9995898008346558], [0.9987549185752869], [0.9993289709091187], [0.0002321302890777588], [0.0005196630954742432], [0.9990575313568115], [0.9995067119598389], [0.00018066167831420898], [0.9995898008346558], [0.00024074316024780273], [0.00019910931587219238], [0.00017341971397399902], [0.9999851584434509], [0.9989343881607056], [0.9998000860214233], [0.9999914765357971], [0.9984145760536194], [0.9995256662368774], [0.9993844032287598], [0.9996182322502136], [0.0004013478755950928], [0.00559428334236145], [0.9994447231292725], [0.0006383955478668213], [6.198883056640625e-06], [0.00017341971397399902], [0.997856616973877], [0.9998611211776733], [0.0002651810646057129], [0.00018066167831420898], [0.0004013478755950928], [0.9999971389770508], [0.00559428334236145], [0.0031388401985168457], [0.9999923706054688], [0.0004210174083709717], [0.6151395440101624], [0.01766502857208252], [5.662441253662109e-07], [0.9994447231292725], [5.662441253662109e-07], [0.9993289709091187], [0.9999964833259

In [33]:
#create list with predictions and parameters from test data
print("Fire Sprinkler only")
new_li=[]
count=0
temp_li=[]
for i in list(test_data)[0][0]['Fire_Sprinkler']:
    temp_li.append(format(i))
    new_li.append(temp_li)
    temp_li=[]
    count+=1
print(new_li)
print("FS and Building Occupancy")
count=0
for i in list(test_data)[0][0]['Building_Occupancy']:
    new_li[count].append(format(i))
    temp_li=[]
    count+=1
print(new_li)
print("FS BS and Room Name")
count=0
for i in list(test_data)[0][0]['Room_Name']:
    new_li[count].append(format(i))
    temp_li=[]
    count+=1
print(new_li)

print("FS BS RM and Room Area")
count=0
for i in list(test_data)[0][0]['numeric']:
    new_li[count].append(float(format(i[0])))
    temp_li=[]
    count+=1
print(new_li)

print("FS BS RM RA and Prediction")
count=0
for i in li:
    if i[0]<0.5:i[0]=0
    else: i[0]=1
    new_li[count].append(i[0])
    temp_li=[]
    count+=1
print(new_li)

Fire Sprinkler only
[['No'], ['Yes'], ['No'], ['Yes'], ['Yes'], ['No'], ['No'], ['Yes'], ['No'], ['Yes'], ['Yes'], ['Yes'], ['No'], ['No'], ['No'], ['No'], ['No'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['No'], ['No'], ['Yes'], ['Yes'], ['No'], ['No'], ['Yes'], ['Yes'], ['Yes'], ['No'], ['Yes'], ['Yes'], ['No'], ['Yes'], ['No'], ['Yes'], ['Yes'], ['No'], ['Yes'], ['No'], ['No'], ['Yes'], ['No'], ['Yes'], ['Yes'], ['Yes'], ['No'], ['No'], ['Yes'], ['No'], ['Yes'], ['No'], ['Yes'], ['No'], ['Yes'], ['Yes'], ['Yes'], ['No'], ['No'], ['No'], ['Yes'], ['No'], ['Yes'], ['No']]
FS and Building Occupancy
[['No', '  Office Building'], ['Yes', '  Medical Building'], ['No', '  Medical Building'], ['Yes', '  Medical Building'], ['Yes', '  Medical Building'], ['No', '  Medical Building'], ['No', '  Apartment Building'], ['Yes', '  Apartment Building'], ['No', '  Office Building'], ['Yes', '  Medical Building'], ['Yes', '  Apartment Building'], ['Yes', '  Office Building'], ['No', '  Apartment

In [36]:
#create pandas dataframe with columns using list and transfer to csv file
CSV_COLUMNS = ['Fire_Sprinkler', 'Building_Occupancy', 'Room_Name', 'Room_Area', 'Smoke_Detector']
new_df=pd.DataFrame(new_li,columns=CSV_COLUMNS)
print(new_df)
new_df.to_csv(r'data-pred.csv')

   Fire_Sprinkler    Building_Occupancy          Room_Name  Room_Area  \
0              No       Office Building             Office      900.0   
1             Yes      Medical Building       Control Room      200.0   
2              No      Medical Building      Nurse Station      200.0   
3             Yes      Medical Building             Office      600.0   
4             Yes      Medical Building      Nurse Station      300.0   
5              No      Medical Building     Operating Room      100.0   
6              No    Apartment Building    Mechanical Room      100.0   
7             Yes    Apartment Building            Storage      300.0   
8              No       Office Building             Office     1000.0   
9             Yes      Medical Building                Gym      300.0   
10            Yes    Apartment Building                Gym      100.0   
11            Yes       Office Building             Office      100.0   
12             No    Apartment Building     Elevato